In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
import tensorflow as tf

In [ ]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2020-07-06 13:30:06--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 172.217.203.138, 172.217.203.113, 172.217.203.139, ...
Connecting to drive.google.com (drive.google.com)|172.217.203.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bhrqiu8vprecsrh8bn6e5snb6g4q84dv/1594042200000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2020-07-06 13:30:07--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bhrqiu8vprecsrh8bn6e5snb6g4q84dv/1594042200000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|1

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
dataset=pd.read_csv('/tmp/sentiment.csv')

In [ ]:
dataset.head()

,Unnamed: 0,text,sentiment
0,0,So there is no way for me to plug it in here i...,0
1,1,Good case Excellent value.,1
2,2,Great for the jawbone.,1
3,3,Tied to charger for conversations lasting more...,0
4,4,The mic is great.,1


In [ ]:
sentences=dataset['text'].tolist()
labels=dataset['sentiment'].tolist()

In [ ]:
training_size=int(len(dataset)*0.8)

training_sentences=sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels=labels[:training_size]
testing_labels=labels[training_size:]


In [ ]:
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [ ]:
tokenizer=Tokenizer(num_words=1000,oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)

word_index=tokenizer.word_index

sequences=tokenizer.texts_to_sequences(training_sentences)

padded=pad_sequences(sequences,padding='post' ,maxlen=100,truncating='post')

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,padding='post' ,maxlen=100,truncating='post')

In [ ]:
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])

In [ ]:
def decode_review(text):
    return " ".join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(1000,16,input_length=100),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1 , activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 16)           16000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 9606      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
num_epochs=20
model.fit(padded,training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded,testing_labels_final))

Epoch 1/20
50/50 [==============================] - 0s 6ms/step - loss: 0.6924 - accuracy: 0.5173 - val_loss: 0.7026 - val_accuracy: 0.4110
Epoch 2/20
50/50 [==============================] - 0s 4ms/step - loss: 0.6862 - accuracy: 0.6089 - val_loss: 0.7057 - val_accuracy: 0.4160
Epoch 3/20
50/50 [==============================] - 0s 4ms/step - loss: 0.6602 - accuracy: 0.6861 - val_loss: 0.6624 - val_accuracy: 0.6065
Epoch 4/20
50/50 [==============================] - 0s 4ms/step - loss: 0.5691 - accuracy: 0.8092 - val_loss: 0.5873 - val_accuracy: 0.7544
Epoch 5/20
50/50 [==============================] - 0s 4ms/step - loss: 0.4370 - accuracy: 0.8688 - val_loss: 0.5079 - val_accuracy: 0.7569
Epoch 6/20
50/50 [==============================] - 0s 4ms/step - loss: 0.3185 - accuracy: 0.9178 - val_loss: 0.5263 - val_accuracy: 0.7419
Epoch 7/20
50/50 [==============================] - 0s 4ms/step - loss: 0.2409 - accuracy: 0.9397 - val_loss: 0.4782 - val_accuracy: 0.7669
Epoch 8/20
50/50 [==

In [ ]:
len(testing_labels_final)

399

In [ ]:
e = model.layers[0]
weights=e.get_weights()[0]

print(weights.shape)

(1000, 16)


In [ ]:
import io

out_v = io.open('vecs.tsv' , 'w' ,encoding='utf-8')
out_m = io.open('meta.tsv' , 'w' ,encoding='utf-8')

for word_num in range(1,1000):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=100)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

I love this phone
[0.99919945]


I hate spaghetti
[0.00405019]


Everything was cold
[0.04240825]


Everything was hot exactly as I wanted
[0.27254862]


Everything was green
[0.26528642]


the host seated us immediately
[0.9876414]


they gave us free chocolate cake
[0.9948159]


not sure about the wilted flowers on the table
[0.00137473]


only works when I stand on tippy toes
[0.9930982]


does not work when I stand on my head
[0.00014521]




In [ ]:
model.summary()

NameError: ignored